In [1]:
import warnings
warnings.filterwarnings('ignore')

### Prepare LLM

In [2]:
import json
import os
from crewai import Agent, Task, Crew

os.environ['OPENAI_API_KEY'] = 'your-openai-api-key'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

### Prepare Tools

In [3]:
from crewai_tools import BaseTool
from app.core.tools.semantic_search_tools import SemanticSearchTool

class SemanticSearchToolForCrewAI(BaseTool):
    name: str ="Search database and table schemas tool"
    description: str = ("Useful to search database and table schemas from vector DB, "
                        "about a given question and return relevant database andtable schemas")
    semantic_search_tool: SemanticSearchTool = None

    def __init__(self, n_results=10, **kwargs):
        super().__init__(**kwargs)
        self.semantic_search_tool = SemanticSearchTool(n_results, **kwargs)

    def _run(self, question: str) -> str:
        return self.semantic_search_tool(question)

# Notes: you can expand n_results to make sure the search results cover final tables, e.g., n_results=50
db_schema_search_tool = SemanticSearchToolForCrewAI(n_results=10)

### Prepare experiment data

In [4]:
from spider_env import SpiderEnv

spider = SpiderEnv(cache_dir='spider')

# select a random question in Spider
observation, info = spider.reset()
question = observation["instruction"]
print('question:', question)
print('gold query:', info['gold_query'])
print('gold result:', info['gold_result'])

Loading cached Spider dataset from spider
Schema file not found for spider/spider/database/epinions_1
Schema file not found for spider/spider/database/small_bank_1
Schema file not found for spider/spider/database/twitter_1
Schema file not found for spider/spider/database/icfp_1
Schema file not found for spider/spider/database/flight_4
Schema file not found for spider/spider/database/company_1
Schema file not found for spider/spider/database/chinook_1
question: Find the famous titles of artists that do not have any volume.
gold query: SELECT Famous_Title FROM artist WHERE Artist_ID NOT IN(SELECT Artist_ID FROM volume)
gold result: [('Antievangelistical Process (re-release)',), ('Antithesis of All Flesh',)]


### Create Agents

In [5]:
with open('app/core/agents/crewai.json') as f:
    agent_config = json.load(f)

search_agent = Agent(**agent_config['assistant_dba'], tools=[db_schema_search_tool])
sql_writer_agent = Agent(**agent_config['senior_sql_writer'])
qa_agent = Agent(**agent_config['senior_qa_engineer'])

### Create Tasks

In [6]:
search_task = Task(
    description='Using search tools to find related database/table candidates, and consolidate all relevant tables that belong the same database for information needed to answer the following question:\n{question}',
    expected_output="Return relevant database and tables' schema.",
    agent=search_agent
)
sql_writer_task = Task(
    description='Based on the database/table schemas, generate a SQL query to answer the following question:\n{question}',
    expected_output="MUST BE ONLY a valid database sql script with required fields, don't add unnecessary fields.",
    agent=sql_writer_agent
)
qa_task = Task(
    description="Check the given sql query carefully, and refine the sql query if necessary to make it fulfill the needs of the following question:\n{question}",
    expected_output="The final sql query. MUST BE ONLY a valid sql script with required fields, don't add unnecessary fields, and must be consistent with table schemas.",
    agent=qa_agent,
    context=[search_task, sql_writer_task]
)

### set-up a Crew

In [7]:
text2sql_crew = Crew(
    agents=[
        search_agent,
        sql_writer_agent,
        qa_agent,
    ],
    tasks=[
        search_task,
        sql_writer_task,
        qa_task,
    ],
    verbose=True
)

### Prepare inputs (question)

In [8]:
text2sql_inputs = {
    'question': question
}

### Kickoff crew actions

In [9]:
result = text2sql_crew.kickoff(inputs=text2sql_inputs)

 [2024-07-20 07:28:10][DEBUG]: == Working Agent: Assistant Search Engineer
 [2024-07-20 07:28:10][INFO]: == Starting Task: Using search tools to find related database/table candidates, and consolidate all relevant tables that belong the same database for information needed to answer the following question:
Find the famous titles of artists that do not have any volume.


> Entering new CrewAgentExecutor chain...
I need to find relevant databases and tables that can help answer the question about famous titles of artists that do not have any volume. I'll start by searching for schemas related to artists, titles, and volumes.

Action: Search database and table schemas tool  
Action Input: {"question": "famous titles of artists that do not have any volume"}   

{"database": "theme_gallery", "table": "artist", "schema": "CREATE TABLE \"artist\" (\n\"Artist_ID\" int,\n\"Name\" text,\n\"Country\" text,\n\"Year_Join\" int,\n\"Age\" int,\nPRIMARY KEY (\"Artist_ID\")\n)"}

{"database": "music_4"

In [10]:
print(result)

```sql
SELECT a.Famous_Title 
FROM artist a 
LEFT JOIN volume v ON a.Artist_ID = v.Artist_ID 
WHERE v.Artist_ID IS NULL;
```


### Compare with gold result

In [11]:
import sqlite3

con = sqlite3.connect('spider/spider/database/music_4/music_4.sqlite')
cursor = con.cursor()
cursor.execute("""SELECT a.Famous_Title 
FROM artist a 
LEFT JOIN volume v ON a.Artist_ID = v.Artist_ID 
WHERE v.Artist_ID IS NULL;""")
results = cursor.fetchall()
print(results)

[('Antievangelistical Process (re-release)',), ('Antithesis of All Flesh',)]


In [12]:
print(info['gold_result'])

[('Antievangelistical Process (re-release)',), ('Antithesis of All Flesh',)]
